In [1]:
from sim.ebm.dy import ModelBaseline
from sim.fn import load_model_baseline
from sim.inputs import load_inputs
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from tqdm.notebook import tqdm
import os

In [2]:
model = load_model_baseline('data')

post = pd.read_csv('pars/baseline/Post.csv').iloc[:, 1:]
post = [dict(row) for _, row in post.iterrows()]
post = [model.Inputs.Cascade.prepare_pars(p) for p in post]

In [3]:
post[0].keys()

dict_keys(['p_txi', 'p_dx', 'p_dx0', 'p_dx1', 'p_ent', 'r_onset', 'r_csi', 'r_recsi', 'mu_a', 'mu_s', 'mu_c', 'tx_alo', 'tx_dur', 'r_txs', 'r_txd', 'r_txl', 'ppv', 'inc', 'prev_ut', 'prev_asc', 'r_die_sym', 'rr_die_asym', 'beta', 'rr_inf_asym', 'rr_sus_slat', 'p_primary', 'r_clear', 'r_sc', 'r_react', 'irr_10', 'irr_20', 'irr_30', 'drt_trans', 'p_txi_pri', 'drt_act', 'rt_cs', 'rr_inf_cs', 'r_lat', 'r_relapse', 'r_relapse_td', 'r_relapse_tc', 't0_decline', 'p_txi_pub', 'p_txi_eng', 'p_dx_pub', 'p_ent_pub', 'r_die_asym', 'r_act'])

In [4]:
mss = list()

for i, p in enumerate(tqdm(post)):
    ys, ms, _ = model.simulate_to_fit(p, np.linspace(2016, 2022, 7))

    y0 = ys.sol(2022)

    _, ms_intv, _ = model.simulate_onward(y0, p)
    mss.append(ms_intv.assign(Scenario='Baseline', Key=i))
    pi = dict(p)
    pi['beta'] = 0
    _, ms_intv, _ = model.simulate_onward(y0, pi)
    mss.append(ms_intv.assign(Scenario='NoBeta', Key=i))
    pi = dict(p)
    pi['r_react'] = 0
    pi['r_relapse'] = pi['r_relapse_td'] = pi['r_relapse_tc'] = 0 
    _, ms_intv, _ = model.simulate_onward(y0, pi)
    mss.append(ms_intv.assign(Scenario='NoAct', Key=i))

mss = pd.concat(mss)

  0%|          | 0/300 [00:00<?, ?it/s]

In [5]:
os.makedirs('out/paf', exist_ok=True)

mss.to_csv('out/paf/mss.csv')